 # **INTRODUCTION**
> In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.
*In this notebook I have used various supervised learning methods to predict which passengers are transported to another world.*

# **IMPORTING DATA**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
space_train=pd.read_csv("/kaggle/input/spaceship-titanic/train.csv")
space_test=pd.read_csv("/kaggle/input/spaceship-titanic/test.csv")
sample=pd.read_csv("/kaggle/input/spaceship-titanic/sample_submission.csv")

# **DATA EXPLORATION**

In [ ]:
space_train.shape

In [ ]:
space_test.shape

In [ ]:
space_train.head()

In [ ]:
space_test.head()

In [ ]:
space_train.info()

In [ ]:
space_train.describe()

In [ ]:
space_train.PassengerId.nunique()/space_train.shape[0]

In [ ]:
space_train.Transported.value_counts()

In [ ]:
space_train.isnull().sum()

In [ ]:
null_value=space_test.isnull().sum()
null_value

In [ ]:
# Replacing the null values 
space_train["HomePlanet"].fillna(method="ffill",inplace=True)
space_train["CryoSleep"].fillna(method="ffill",inplace=True)
space_train["Cabin"].fillna(method="ffill",inplace=True)
space_train["Destination"].fillna(method="ffill",inplace=True)
space_train["Age"].fillna(space_train["Age"].mean(),inplace=True)
space_train["VIP"].fillna(method="ffill",inplace=True)
space_train["RoomService"].fillna(space_train["RoomService"].mean(),inplace=True)
space_train["FoodCourt"].fillna(space_train["FoodCourt"].mean(),inplace=True)
space_train["ShoppingMall"].fillna(space_train["ShoppingMall"].mean(),inplace=True)
space_train["Spa"].fillna(space_train["Spa"].mean(),inplace=True)
space_train["VRDeck"].fillna(space_train["VRDeck"].mean(),inplace=True)
space_train["Name"].fillna(method="ffill",inplace=True)


In [ ]:
# Rechecking after eliminating null values
space_train.isnull().sum()

In [ ]:
# Replacing the null values 
space_test["HomePlanet"].fillna(method="ffill",inplace=True)
space_test["CryoSleep"].fillna(method="ffill",inplace=True)
space_test["Cabin"].fillna(method="ffill",inplace=True)
space_test["Destination"].fillna(method="ffill",inplace=True)
space_test["Age"].fillna(space_test["Age"].mean(),inplace=True)
space_test["VIP"].fillna(method="ffill",inplace=True)
space_test["RoomService"].fillna(space_test["RoomService"].mean(),inplace=True)
space_test["FoodCourt"].fillna(space_test["FoodCourt"].mean(),inplace=True)
space_test["ShoppingMall"].fillna(space_test["ShoppingMall"].mean(),inplace=True)
space_test["Spa"].fillna(space_test["Spa"].mean(),inplace=True)
space_test["VRDeck"].fillna(space_test["VRDeck"].mean(),inplace=True)
space_test["Name"].fillna(method="ffill",inplace=True)

In [ ]:
# Rechecking after eliminating null values
space_test.isnull().sum()

In [ ]:
plt.figure(figsize=(5, 5))
space_train["Age"].plot.box(vert=False)
space_train=space_train[(space_train["Age"]<=60)]
plt.figure(figsize=(5, 5))
space_train["Age"].plot.box(vert=False)

In [ ]:
expenses=["FoodCourt","ShoppingMall","Spa","VRDeck","RoomService"]
space_train.boxplot(column=expenses)
q_value =[i/100 for i in range (95,101,1)]
q1=space_train[expenses].quantile(q_value)
q1


In [ ]:
space_train=space_train[(space_train["FoodCourt"]<=7993.12) & (space_train["ShoppingMall"]<=2319.92)& (space_train["Spa"]<=5255.60)& (space_train["VRDeck"]<=5577.56)& (space_train["RoomService"]<=3036.28)]
space_train.shape
space_train.boxplot(column=expenses)
space_train=space_train[(space_train["FoodCourt"]<=7993.12) & (space_train["ShoppingMall"]<=2319.92)& (space_train["Spa"]<=5255.60)& (space_train["VRDeck"]<=5577.56)& (space_train["RoomService"]<=3036.28)]
print(space_train.shape)
space_train.boxplot(column=expenses)

# **DATA VISUALIZATION**

In [ ]:
plt.figure(figsize=(7,7))
plt.title("Distribution of Transported Passengers")
space_train["Transported"].value_counts().plot(kind="pie",autopct='%1.2f%%')

There is not much difference in the distribution of Transported and non transported passengers. Hence, the data is unbaised. 

In [ ]:
plt.figure(figsize=(7,7))
sns.barplot(x="Transported",y="HomePlanet",data=space_train)
plt.title("Transported successfully from Home Planet")

Maximum no of passengers transported are from planet Eupora

In [ ]:
plt.figure(figsize=(7,7))
plt.title(" Passengers Confined to Cabins")
space_train["CryoSleep"].value_counts().plot(kind="pie",autopct='%1.2f%%')

In [ ]:
plt.figure(figsize=(7,7))
sns.barplot(x="Transported",y="CryoSleep",data=space_train)
plt.title("Confined Passengers Transported")

Maximum confined passengers are transported.

In [ ]:
plt.figure(figsize=(7,7))
space_train["Destination"].value_counts().plot(kind="pie",autopct='%1.2f%%')

In [ ]:
plt.figure(figsize=(7,7))
sns.barplot(x="Transported",y="Destination",data=space_train)

Maximum transported to 55 Cancri e

In [ ]:
# splitting the cabin to Side : Port(P)  or Standard(S)
space_train["Side"]=space_train["Cabin"].str.split("/").str[2] 
space_test["Side"]=space_test["Cabin"].str.split("/").str[2] 
plt.figure(figsize=(7,7))
space_train["Side"].value_counts().plot(kind="pie",autopct='%1.2f%%')

In [ ]:
plt.figure(figsize=(7,7))
sns.catplot(x="Side",y="Transported",kind="bar",palette="mako",data=space_train)

In [ ]:
space_train["Deck"]=space_train["Cabin"].str.split("/").str[0] 
space_test["Deck"]=space_test["Cabin"].str.split("/").str[0] 
plt.figure(figsize=(7,7))
space_train["Deck"].value_counts().plot(kind="pie",autopct='%1.2f%%')

In [ ]:
plt.figure(figsize=(7,7))
sns.catplot(x="Transported",y="Deck",kind="bar",palette="ch:.25",data=space_train)

In [ ]:
plt.figure(figsize=(10,10))
sns.catplot(x="HomePlanet",y="Transported",hue="Destination",kind="bar",palette="pastel",data=space_train)

In [ ]:
plt.figure(figsize=(10,10))
sns.catplot(x="HomePlanet",y="Age",hue="Transported",kind="box",palette="viridis",data=space_train)

In [ ]:
plt.figure(figsize=(7,7))
space_train["VIP"].value_counts().plot(kind="pie",autopct='%1.2f%%')

In [ ]:
plt.figure(figsize=(10,10))
sns.catplot(x="Destination",y="VIP",hue="Transported",kind="point",palette="Spectral",data=space_train)

In [ ]:
space_train["Expenses"]=space_train["RoomService"]+space_train["FoodCourt"]+space_train["ShoppingMall"]+space_train["Spa"]+space_train["VRDeck"]
plt.figure(figsize=(10,10))
sns.catplot(x="VIP",y="Expenses",hue="Transported",kind="bar",palette="icefire",data=space_train)

In [ ]:
plt.figure(figsize=(10,10))
sns.catplot(x="HomePlanet",y="Expenses",hue="Transported",kind="bar",palette="coolwarm",data=space_train)

In [ ]:
sns.scatterplot(x='Age',y='Expenses',data=space_train[space_train.Transported==True])

In [ ]:
plt.figure(figsize=(7,7))
sns.barplot(x="Side",y="Expenses",palette="ch:s=-.2,r=.6",data=space_train)

# **DATA ENCODING**

In [ ]:
space_train.head()

In [ ]:
space_test.head()

In [ ]:
space_train.drop(["PassengerId","Name","Expenses","Cabin"],inplace=True,axis=1)
space_test.drop(["PassengerId","Name","Cabin"],inplace=True,axis=1)

In [ ]:
space_train.head()

In [ ]:
space_test.head()

In [ ]:
categorical_var=[i for i in space_train.columns if space_train[i].dtypes=='object']
for z in categorical_var:
    print(space_train[z].name,':',space_train[z].unique())

In [ ]:
numerical_var=space_train[[i for i in space_train.columns if space_train[i].dtypes !='object']]
print(numerical_var)
numerical_var.plot(subplots=True, figsize=(8, 8))

In [ ]:
categorical_var=[i for i in space_test.columns if space_test[i].dtypes=='object']
for z in categorical_var:
    print(space_test[z].name,':',space_test[z].unique())

In [ ]:
numerical_var=space_test[[i for i in space_test.columns if space_test[i].dtypes !='object']]
numerical_var
numerical_var.plot(subplots=True, figsize=(8, 8))

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
for x in [i for i in space_train.columns if len(space_train[i].unique())==2]:
    space_train[x]= label_encoder.fit_transform(space_train[x])
    

In [ ]:
#Encode Variables with more than 2 Classes
space_train= pd.get_dummies(space_train, columns= [i for i in space_train.columns if space_train[i].dtypes=='object'],drop_first=True)


In [ ]:
label_encoder=LabelEncoder()
for x in [i for i in space_test.columns if len(space_test[i].unique())==2]:
    space_test[x]= label_encoder.fit_transform(space_test[x])

In [ ]:
#Encode Variables with more than 2 Classes
space_test= pd.get_dummies(space_test, columns= [i for i in space_test.columns if space_test[i].dtypes=='object'],drop_first=True)

In [ ]:
space_train.shape

In [ ]:
space_test.shape

# **SPLITTING DATA**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix , classification_report,accuracy_score
from sklearn.metrics import roc_curve , auc
from sklearn.preprocessing import StandardScaler

In [ ]:
## Defining variables X,y 
x_train= space_train.drop(["Transported"],axis=1)
x_test=space_test
Y_train=space_train["Transported"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x_train, Y_train, test_size = 0.25, random_state = 0)

In [ ]:
print("shape of X_train:",X_train.shape)
print("shape of y_train:",y_train.shape[0])

In [ ]:
space_train.head()

In [ ]:
## Scaling the data 
sc= StandardScaler()
X_train = sc.fit_transform(X_train)
X_test=sc.transform(X_test)
X_train=pd.DataFrame(X_train,columns=x_train.columns)
X_test=pd.DataFrame(X_test,columns=x_test.columns)

# **MODEL PREDICTION**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import model_selection
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [ ]:
# prepare configuration for cross validation test harness
seed = 7
# prepare models
models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('RFC',RandomForestClassifier(max_depth=20,random_state=200)))
models.append
# evaluate each model in turn
results = []
names = []
scoring = 'accuracy'
for name, model in models:
    kfold = model_selection.KFold(n_splits=10)
    cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)
# boxplot algorithm comparison
fig = plt.figure()
fig.suptitle('Algorithm Comparison')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

The RFC has highest accuracy rate.

# **MODEL SELECTION**


In [ ]:
model=RandomForestClassifier(max_depth=100,random_state=42)
model.fit(X_train,y_train)

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
## confusion matrix
conf_mat=confusion_matrix(y_test,y_pred)
print("Confusion matrix is \n",conf_mat)

## plotting confusion matrix
plt.figure(figsize=(7,7))
plt.title("CM")
sns.heatmap(conf_mat, square=True,cmap="BuPu",annot=True,fmt='d')
plt.xlabel('true label')
plt.ylabel('predicted label')

In [ ]:
y_pred_proba = model.predict_proba(X_test)[:,1]

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
roc_auc=auc(fpr,tpr)
#Now Draw ROC using fpr , tpr
plt.plot([0, 1], [0, 1], 'k--',label='Random')
plt.plot(fpr,tpr,label='ROC curve (area = %0.2f)' %roc_auc)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title(" ROC curve")
plt.legend(loc='best')

In [ ]:
## Distribution of Transported Passengers
plt.figure(figsize=(10,10))
plt.title("Distribution of Transported Passengers")
y_test.value_counts().plot(kind="pie",autopct='%1.2f%%')

# **SMOTE**

In [ ]:
from imblearn.over_sampling import SMOTE
smote=SMOTE()
smote_X,smote_y=smote.fit_resample(X_train,y_train)

In [ ]:
## Distribution of heart diseases
plt.figure(figsize=(10,10))
plt.title("Distribution of Transported Passengers")
smote_y.value_counts().plot(kind="pie",autopct='%1.2f%%')

In [ ]:
model_smote=RandomForestClassifier(max_depth=100,random_state=42)
model_smote.fit(smote_X,smote_y)

In [ ]:
y_pred_smote=model_smote.predict(X_test)

In [ ]:
## classification report
print(classification_report(y_test,y_pred_smote))

In [ ]:
## confusion matrix
conf_mat_smote=confusion_matrix(y_test,y_pred_smote)
print("Confusion matrix is \n",conf_mat_smote)

## plotting confusion matrix

plt.figure(figsize=(7,7))
plt.title(" SMOTE CM")
sns.heatmap(conf_mat_smote, square=True,cmap="Blues",annot=True,fmt='d')
plt.xlabel("Predicted Value")
plt.ylabel("Actual value")

In [ ]:
print("accuracy:", round(100*accuracy_score(y_test,y_pred_smote)),"%")

In [ ]:
y_pred_proba_smote = model_smote.predict_proba(X_test)[:,1]

In [ ]:
fpr, tpr, _ = roc_curve(y_test, y_pred_proba_smote)
roc_auc=auc(fpr,tpr)
#Now Draw ROC using fpr , tpr
plt.plot([0, 1], [0, 1], 'k--',label='Random')
plt.plot(fpr,tpr,label='ROC curve (area = %0.2f)' %roc_auc)
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('SMOTE ROC curve')
plt.legend(loc='best')

# **FEATURES**

In [ ]:
xx= pd.Series(model.feature_importances_,index=x_train.columns)
plt.figure(figsize =(5,10))
plt.title("Feature importance plot")
xx.nlargest(10).sort_values(ascending=True).plot(kind="barh",)


In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
fit = pca.fit(x_train)
# summarize components
print("Explained Variance: %s" % fit.explained_variance_ratio_)
print(fit.components_)

# **SUBMISSION**

In [ ]:
y_final=model.predict(x_test)

In [ ]:
sub=pd.DataFrame({'PassengerId':sample["PassengerId"],"Transported":y_final.astype('bool')})

In [ ]:
sub.to_csv('submission.csv',index=False)
pd.read_csv('submission.csv')